## Lightweight python components

In the last notebook, we show you how to build a pipeline using a exisitng container image. Lightweight python components do not require you to build a new container image for every code change. They're intended to use for fast iteration in notebook environment.


## Building a lightweight python component

There are several requirements for the function:

- The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
- The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package. There is an example below in my_divmod function.)
- If the function operates on numbers, the parameters need to have type hints. Supported types are [int, float, bool]. Everything else is passed as string.
- To build a component with multiple output values, use the typing.NamedTuple type hint syntax: NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])


### Import kfp library

In [ ]:
import kfp
from kfp import components

### Create a Python function to wrap your component


In [ ]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

### Convert the fucntion to a pipeline operation

In [ ]:
add_op = components.func_to_container_op(add, base_image='python:3.6.8')

## Define your pipeline as a Python function


In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calculate_sum_lightweight(
   a='a',
   b='7',
   c='17',
   d='4'
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task1 = add_op(a, b) #Returns a dsl.ContainerOp class instance. 
    add_task2 = add_op(c, d) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    result_task = add_op(add_task1.output, add_task2.output)

## Compile the pipeline


In [ ]:
kfp.compiler.Compiler().compile(calculate_sum_lightweight,'calculate-sum-pipeline-lightweight.zip')

## Deploy the pipeline

Download `calculate-sum-pipeline-lightweight.zip` from your notebook server.
![download-pipeline](./images/download_pipeline.jpg)

Upload the generated `calculate-sum-pipeline-lightweight.zip` file through the Kubeflow Pipelines UI.

Please follow [instructions](https://www.kubeflow.org/docs/gke/pipelines-tutorial/#run-the-pipeline) to run your pipeline 